Week 5, Assignment C1:

Logistic Regression with Yahoo! Finance API in Python

In this notebook, we will study Logistic Regression using the Yahoo! Finance API in Python.

Please complete the lines bellow where you see "#TODO".

In [1]:
#Objective 1.
#Use this cell to import the Numpy (as np), Pandas (as pd), and YFinance (as yf) packages.

import pandas as pd
import numpy as np 
import yfinance as yf

#Our work will also require some components of the Sklearn and Pandas_Datareader packages as imported below:

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from pandas_datareader import data as pdr
yf.pdr_override()

In [2]:
#Objective 2.
#Select a stock symbol for a stock whose historical data is available on the Yahoo! Finance website.  Store the
#string of your chosen symbol to the new variable "stock_symbol".

stock_symbol = '2222.SR'
#aramco

In [3]:
#Objective 3.
#Use the function "pdr.get_data_yahoo(stock_symbol, start_date, end_date)" to generate a Pandas dataframe of
#historical stock data for your chosen stock.  Retain only the first four columns of the dataframe.  Be sure to 
#drop any rows containing NaN's, and take a peek at the resulting dataframe to make sure everything looks good.

df = pdr.get_data_yahoo( stock_symbol , '2021-01-01', '2021-11-07')
df = df.iloc[:, :4]
df = df.dropna()
df.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close
Date,,,,
2021-01-03,35.000000,35.000000,34.849998,34.849998
2021-01-04,34.849998,35.000000,34.799999,34.849998
2021-01-05,34.900002,34.900002,34.650002,34.849998
2021-01-06,34.849998,34.950001,34.700001,34.799999
2021-01-07,34.750000,34.900002,34.700001,34.900002


In [4]:
#Objective 4.
#Use the historical data to define some predictor variables.  Add these variables to the dataframe.  Include, at a
#minimum, (Predictor.I) the rolling average closing price over the past fifteen (15) days and (Predictor.II) the
#change in opening price over the past one (1) day.  Be sure to again drop any rows containing NaN's, and take a
#peek at the resulting dataframe to make sure everything looks good.  Store the dataframe as the new variable "X".

df['Close_15_Rolling'] = df['Close'].rolling(window = 15).mean()
df['Open_1_Change'] = df['Open'] - df['Open'].shift(1)
df = df.dropna()
X = df
X

,Open,High,Low,Close,Close_15_Rolling,Open_1_Change
Date,,,,,,
2021-01-21,34.799999,34.900002,34.799999,34.799999,34.933333,0.000000
2021-01-24,34.799999,34.950001,34.799999,34.950001,34.940000,0.000000
2021-01-25,34.849998,34.950001,34.750000,34.750000,34.933334,0.049999
2021-01-26,34.750000,34.900002,34.700001,34.700001,34.923334,-0.099998
2021-01-27,34.750000,34.849998,34.700001,34.799999,34.923334,0.000000
...,...,...,...,...,...,...
2021-10-31,37.900002,38.299999,37.799999,37.900002,37.400001,0.300003
2021-11-01,37.900002,38.000000,37.750000,37.750000,37.453334,0.000000
2021-11-02,37.750000,37.900002,37.650002,37.900002,37.500001,-0.150002


In [5]:
#Objective 5.
#Define the target or dependent variable to be one (1) if the change in closing price over the past one (1) day is
#nonnegative and negative one (-1) if the change in closing price over the past one (1) day is negative.  This
#variable's values should be forward-looking (i.e., you should subtract today's price from tomorrow's price rather
#than subtracting yesterday's price from today's price).  Store the resulting values as the new variable "y".  You
#may find the "np.where(*args)" function to be useful.  Look it up in Numpy documentation for support.

y = np.where(df['Close'].shift(1) - df['Close'] >= 0, 1 , -1)

In [6]:
#Objective 6.
#Split the data into training and test sets, putting the first seventy percent (70%) of the data in the training
#set.
from sklearn.model_selection import train_test_split
index = int(0.7*len(X))
X_train, X_test, y_train, y_test = train_test_split(X , y , train_size= int(0.7*len(X)))

In [7]:
#Objective 7.
#Instantiate the Logistic Regression model object, and use its ".fit(*args)" method to fit the model to the 
#training data.

logistic = LogisticRegression()
logistic = logistic.fit(X_train , y_train)

In [8]:
#Objective 8.
#Examine the model's coefficients by using its ".coef_" method.

pd.DataFrame(zip(X.columns, np.transpose(logistic.coef_)))

,0,1
0,Open,[2.484233886003268]
1,High,[-1.1769890535100487]
2,Low,[0.8987593692366658]
3,Close,[-2.5075425754173843]
4,Close_15_Rolling,[0.2543191386009114]
5,Open_1_Change,[0.05632878293624444]


In [9]:
#Objective 9.
#Use the model's ".predict_proba(*args)" and ".predict(*args)" methods to generate predictions over the test set.

probabilities = logistic.predict_proba(X_test)
predictions = logistic.predict(X_test)

In [10]:
#Objective 10.
#Use the function "metrics.confusion_matrix(*args)" to create a confusion matrix comparing the predicted and true
#classification labels over the test set.

from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test ,predictions)
print(matrix)

[[ 3 17]
 [ 0 39]]


In [11]:
#Objective 11.
#Calculate the model's accuracy on the test set using its ".score(*args)" method.

from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

0.711864406779661

In [18]:
#Objective 12.
#Use five-fold cross validation to cross-check the accuracy of the model over different held-out test sets.  This
#is where you should use the function "cross_val_score(*args)".

from sklearn.model_selection import cross_val_score
cross_val = cross_val_score(LogisticRegression() ,X, y, scoring = 'accuracy' , cv=5)
print(cross_val)
print(cross_val.mean())

[0.65       0.69230769 0.64102564 0.71794872 0.66666667]
0.6735897435897436


C:\Users\RW\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
